In [1]:
import requests
import pandas as pd
import json
from dateutil import parser

In [2]:
API_KEY = "41e931078da8d27b7e4befaab7450b98-4707abfc0f53775372778340d7fcda02"
ACCOUNT_ID = "101-001-26602725-001"
OANDA_URL = "https://api-fxpractice.oanda.com/v3"

In [3]:
#נפתח סשן שממנו נבקש את כל השיט
#הפואנטה בסשן-אפשר להגדיר "כותרות" (headers) שהם קבועות וצריך אותן בכל בקשה

In [4]:
session = requests.Session()

In [5]:
session.headers.update({"Authorization": f"Bearer {API_KEY}",
                        "Content-Type": "application/json"
                       })

In [6]:
params = dict(
    count = 10,
    granularity = "H1",
    price = "MBA"
)

In [7]:
url= f"{OANDA_URL}/accounts/{ACCOUNT_ID}/instruments"

In [8]:
response = session.get(url, params = None, data=None, headers=None)

In [9]:
response.status_code

200

In [10]:
data = response.json()

In [11]:
print(json.dumps(data, indent=2))

{
  "instruments": [
    {
      "name": "USD_SGD",
      "type": "CURRENCY",
      "displayName": "USD/SGD",
      "pipLocation": -4,
      "displayPrecision": 5,
      "tradeUnitsPrecision": 0,
      "minimumTradeSize": "1",
      "maximumTrailingStopDistance": "1.00000",
      "minimumTrailingStopDistance": "0.00050",
      "maximumPositionSize": "0",
      "maximumOrderUnits": "100000000",
      "marginRate": "0.05",
      "guaranteedStopLossOrderMode": "DISABLED",
      "tags": [
        {
          "type": "BRAIN_ASSET_CLASS",
          "name": "FX"
        },
        {
          "type": "ASSET_CLASS",
          "name": "CURRENCY"
        }
      ],
      "financing": {
        "longRate": "0.0052",
        "shortRate": "-0.0299",
        "financingDaysOfWeek": [
          {
            "dayOfWeek": "MONDAY",
            "daysCharged": 1
          },
          {
            "dayOfWeek": "TUESDAY",
            "daysCharged": 1
          },
          {
            "dayOfWeek": "WED

In [12]:
data.keys()

dict_keys(['instruments', 'lastTransactionID'])

In [13]:
instruments_list = data['instruments']
len(instruments_list)

68

In [14]:
print(json.dumps(instruments_list[0], indent=2))

{
  "name": "USD_SGD",
  "type": "CURRENCY",
  "displayName": "USD/SGD",
  "pipLocation": -4,
  "displayPrecision": 5,
  "tradeUnitsPrecision": 0,
  "minimumTradeSize": "1",
  "maximumTrailingStopDistance": "1.00000",
  "minimumTrailingStopDistance": "0.00050",
  "maximumPositionSize": "0",
  "maximumOrderUnits": "100000000",
  "marginRate": "0.05",
  "guaranteedStopLossOrderMode": "DISABLED",
  "tags": [
    {
      "type": "BRAIN_ASSET_CLASS",
      "name": "FX"
    },
    {
      "type": "ASSET_CLASS",
      "name": "CURRENCY"
    }
  ],
  "financing": {
    "longRate": "0.0052",
    "shortRate": "-0.0299",
    "financingDaysOfWeek": [
      {
        "dayOfWeek": "MONDAY",
        "daysCharged": 1
      },
      {
        "dayOfWeek": "TUESDAY",
        "daysCharged": 1
      },
      {
        "dayOfWeek": "WEDNESDAY",
        "daysCharged": 1
      },
      {
        "dayOfWeek": "THURSDAY",
        "daysCharged": 1
      },
      {
        "dayOfWeek": "FRIDAY",
        "daysCha

In [15]:
instruments_list[0].keys()

dict_keys(['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'minimumTradeSize', 'maximumTrailingStopDistance', 'minimumTrailingStopDistance', 'maximumPositionSize', 'maximumOrderUnits', 'marginRate', 'guaranteedStopLossOrderMode', 'tags', 'financing'])

In [16]:
interesting_keys=['name', 'type', 'displayName', 'pipLocation',
                  'displayPrecision', 'tradeUnitsPrecision', 'marginRate']

In [17]:
instruments_dict = {}
for i in instruments_list:
    key = i['name']
    instruments_dict[key] = { k: i[k] for k in interesting_keys }

In [18]:
print(json.dumps(instruments_dict, indent=2))

{
  "USD_SGD": {
    "name": "USD_SGD",
    "type": "CURRENCY",
    "displayName": "USD/SGD",
    "pipLocation": -4,
    "displayPrecision": 5,
    "tradeUnitsPrecision": 0,
    "marginRate": "0.05"
  },
  "EUR_SEK": {
    "name": "EUR_SEK",
    "type": "CURRENCY",
    "displayName": "EUR/SEK",
    "pipLocation": -4,
    "displayPrecision": 5,
    "tradeUnitsPrecision": 0,
    "marginRate": "0.03"
  },
  "HKD_JPY": {
    "name": "HKD_JPY",
    "type": "CURRENCY",
    "displayName": "HKD/JPY",
    "pipLocation": -4,
    "displayPrecision": 5,
    "tradeUnitsPrecision": 0,
    "marginRate": "0.1"
  },
  "AUD_USD": {
    "name": "AUD_USD",
    "type": "CURRENCY",
    "displayName": "AUD/USD",
    "pipLocation": -4,
    "displayPrecision": 5,
    "tradeUnitsPrecision": 0,
    "marginRate": "0.03"
  },
  "USD_CAD": {
    "name": "USD_CAD",
    "type": "CURRENCY",
    "displayName": "USD/CAD",
    "pipLocation": -4,
    "displayPrecision": 5,
    "tradeUnitsPrecision": 0,
    "marginRate": "

In [19]:
with open("../data/instruments.json", "w") as f:
    f.write(json.dumps(instruments_dict, indent=2))

In [20]:
#25-Fetching candles1
#פונקציה שתחזיר נרות על פי פרמטרים שאעביר לה
#הEND POINT הזה הרי מחזיר מילון עם 3 מפתחות, זה שאנחנו מעוניינים בו זה מפתח הCANDLES.
#הפואנטה של הפונקציה זה להביא לנו את הנרות בלבד (עם הפרמטרים שקיבלנו בפונקציה)

In [21]:
def fetch_candles(pairName, count=10, granularity="H1"):
    
    url = f"{OANDA_URL}/instruments/{pairName}/candles"
    
    params = dict(
        count = count,
        granularity = granularity,
        price = "MBA"
    )
    
    response = session.get(url, params = params, data=None, headers=None)
    data =response.json()

    if response.status_code == 200:
        if 'candles' not in data:
            data=[]
        else:
            data=data['candles']
        return response.status_code, data

In [22]:
code, data = fetch_candles("USD_CAD", count=5)

In [23]:
print(json.dumps(data, indent=2))

[
  {
    "complete": true,
    "volume": 2409,
    "time": "2023-10-04T05:00:00.000000000Z",
    "bid": {
      "o": "1.36968",
      "h": "1.37095",
      "l": "1.36940",
      "c": "1.37058"
    },
    "mid": {
      "o": "1.36978",
      "h": "1.37104",
      "l": "1.36951",
      "c": "1.37068"
    },
    "ask": {
      "o": "1.36989",
      "h": "1.37114",
      "l": "1.36962",
      "c": "1.37077"
    }
  },
  {
    "complete": true,
    "volume": 4443,
    "time": "2023-10-04T06:00:00.000000000Z",
    "bid": {
      "o": "1.37058",
      "h": "1.37184",
      "l": "1.37038",
      "c": "1.37146"
    },
    "mid": {
      "o": "1.37068",
      "h": "1.37193",
      "l": "1.37049",
      "c": "1.37156"
    },
    "ask": {
      "o": "1.37078",
      "h": "1.37202",
      "l": "1.37060",
      "c": "1.37165"
    }
  },
  {
    "complete": true,
    "volume": 6036,
    "time": "2023-10-04T07:00:00.000000000Z",
    "bid": {
      "o": "1.37148",
      "h": "1.37231",
      "l": "1.3

In [25]:
#מעבר לפורמט של פנדס
#נתחיל בליצוור טבלה שהעמודות שלה זה הווליום והזמן ונסנן מתוכה רק את הנרות שכבר סיימו

In [26]:
final_data =[]
for candle in data:
    if candle['complete'] == False:
        continue
    new_dict = {}
    new_dict['time'] = parser.parse(candle['time'])
    new_dict['volume'] = candle['volume']
    final_data.append(new_dict)
df = pd.DataFrame.from_dict(final_data)  #ככה ממירים מילון לפנדה

In [27]:
df

,time,volume
0,2023-08-16 11:00:00+00:00,4290
1,2023-08-16 12:00:00+00:00,5761
2,2023-08-16 13:00:00+00:00,8656
3,2023-08-16 14:00:00+00:00,8634


In [28]:
#הוספה של המחירים

In [29]:
prices = ['bid', 'mid', 'ask']
ohlc = ['o', 'h', 'l', 'c']

In [30]:
final_data =[]
for candle in data:
    if candle['complete'] == False:
        continue
    new_dict = {}
    new_dict['time'] = parser.parse(candle['time'])
    new_dict['volume'] = candle['volume']

    for p in prices:
        for o in ohlc:
            new_dict[f"{p}_{o}"] = float(candle[p][o])
    
    final_data.append(new_dict)
df = pd.DataFrame.from_dict(final_data)  

In [31]:
df

,time,volume,bid_o,bid_h,bid_l,bid_c,mid_o,mid_h,mid_l,mid_c,ask_o,ask_h,ask_l,ask_c
0,2023-08-16 11:00:00+00:00,4290,1.34949,1.34983,1.34879,1.34893,1.34958,1.34993,1.34888,1.34902,1.34966,1.35003,1.34896,1.34912
1,2023-08-16 12:00:00+00:00,5761,1.34893,1.35059,1.34860,1.35054,1.34902,1.35068,1.34870,1.35062,1.34911,1.35077,1.34879,1.35069
2,2023-08-16 13:00:00+00:00,8656,1.35056,1.35068,1.34809,1.34981,1.35064,1.35078,1.34819,1.34990,1.35071,1.35087,1.34829,1.35000
3,2023-08-16 14:00:00+00:00,8634,1.34985,1.35262,1.34981,1.35148,1.34995,1.35270,1.34990,1.35158,1.35005,1.35279,1.34999,1.35167


In [32]:
#נהפוך לפונקציה
#מקבל- דאטה של נרות נטו
#מחזיר- פנדה עם העמודות זמן מחיר ונפח

In [52]:
def get_candles_df(data):
    if len(data) == 0:
        return pd.DataFrame()

    final_data = []
    for candle in data:
        if candle['complete'] == False:
            continue
        new_dict = {}
        new_dict['time'] = parser.parse(candle['time'])
        new_dict['volume'] = candle['volume']

        for p in prices:
            for o in ohlc:
                new_dict[f"{p}_{o}"] = float(candle[p][o])
    
        final_data.append(new_dict)
    df = pd.DataFrame.from_dict(final_data)  
    return df


In [53]:
code, data1= fetch_candles("EUR_USD", count=10, granularity="H4")
candles_df = get_candles_df(data1)
candles_df

,time,volume,bid_o,bid_h,bid_l,bid_c,mid_o,mid_h,mid_l,mid_c,ask_o,ask_h,ask_l,ask_c
0,2023-08-15 01:00:00+00:00,14348,1.09067,1.09168,1.08997,1.09092,1.09074,1.09177,1.09005,1.09102,1.09082,1.09186,1.09013,1.09111
1,2023-08-15 05:00:00+00:00,21720,1.09092,1.09336,1.09054,1.09272,1.09100,1.09344,1.09063,1.09282,1.09109,1.09352,1.09072,1.09292
2,2023-08-15 09:00:00+00:00,24893,1.09273,1.09443,1.08956,1.09210,1.09282,1.09450,1.08968,1.09218,1.09292,1.09457,1.08977,1.09225
3,2023-08-15 13:00:00+00:00,30912,1.09210,1.09517,1.09144,1.09172,1.09218,1.09524,1.09151,1.09180,1.09226,1.09532,1.09158,1.09187
4,2023-08-15 17:00:00+00:00,10821,1.09171,1.09204,1.08983,1.09035,1.09179,1.09211,1.08991,1.09044,1.09187,1.09218,1.08999,1.09054
5,2023-08-15 21:00:00+00:00,5858,1.09035,1.09060,1.08985,1.09025,1.09085,1.09106,1.08992,1.09034,1.09135,1.09156,1.09000,1.09042
6,2023-08-16 01:00:00+00:00,15078,1.09022,1.09128,1.08982,1.09127,1.09030,1.09136,1.08990,1.09135,1.09038,1.09144,1.08998,1.09143
7,2023-08-16 05:00:00+00:00,19988,1.09125,1.09340,1.09098,1.09273,1.09134,1.09347,1.09106,1.09282,1.09143,1.09354,1.09114,1.09290
8,2023-08-16 09:00:00+00:00,17973,1.09273,1.09319,1.09016,1.09016,1.09280,1.09327,1.09024,1.09024,1.09288,1.09335,1.09031,1.09031


In [54]:
#SUMMARY OF THE FUNCTIONS BY NOW

In [59]:
def fetch_candles(pair_name, count=10, granularity="H1"):
    url = f"{OANDA_URL}/instruments/{pair_name}/candles"
    params = dict(
        count = count,
        granularity = granularity,
        price = "MBA"
    )
    response = session.get(url, params=params, data=None, headers=None)
    data = response.json()
    
    if response.status_code == 200:
        if 'candles' not in data:
            data = []
        else:
            data = data['candles']
    return response.status_code, data

def get_candles_df(data):
    if len(data) == 0:
        return pd.DataFrame.empty
    
    prices = ['mid', 'bid', 'ask']
    ohlc = ['o', 'h', 'l', 'c']
    
    final_data = []
    for candle in data:
        if candle['complete'] == False:
            continue
        new_dict = {}
        new_dict['time'] = parser.parse(candle['time'])
        new_dict['volume'] = candle['volume']
        for p in prices:
            for o in ohlc:
                new_dict[f"{p}_{o}"] = float(candle[p][o])
        final_data.append(new_dict)
    df = pd.DataFrame.from_dict(final_data)
    return df

def create_data_file(pair_name, count=10, granularity="H1"):
    code, data = fetch_candles(pair_name, count, granularity)
    if code != 200:
        print("Failed", pair_name, data)
        return
    if len(data) == 0:
        print("No candles", pair_name)
    candles_df = get_candles_df(data)
    candles_df.to_pickle(f"../data/{pair_name}_{granularity}.pkl")
    print(f"{pair_name} {granularity} {candles_df.shape[0]} candles, {candles_df.time.min()} {candles_df.time.max()}")
    

In [60]:
 create_data_file("EUR_USD", count = 10, granularity="H4")

EUR_USD H4 9 candles, 2023-08-15 01:00:00+00:00 2023-08-16 09:00:00+00:00


In [61]:
our_curr = ['EUR', 'USD', 'GBP', 'JPY', 'CHF', 'NZD','CAD', 'AUD']

for i in our_curr:
    for j in our_curr:
        pair = f"{i}_{j}"
        if pair in instruments_dict.keys():
            for g in ["H1", "H4"]:
                create_data_file(pair, count=4001, granularity = g) 


EUR_USD H1 4000 candles, 2022-12-26 23:00:00+00:00 2023-08-16 14:00:00+00:00
EUR_USD H4 4000 candles, 2021-01-24 22:00:00+00:00 2023-08-16 09:00:00+00:00
EUR_GBP H1 4000 candles, 2022-12-26 23:00:00+00:00 2023-08-16 14:00:00+00:00
EUR_GBP H4 4000 candles, 2021-01-24 22:00:00+00:00 2023-08-16 09:00:00+00:00
EUR_JPY H1 4000 candles, 2022-12-26 23:00:00+00:00 2023-08-16 14:00:00+00:00
EUR_JPY H4 4000 candles, 2021-01-22 14:00:00+00:00 2023-08-16 09:00:00+00:00
EUR_CHF H1 4000 candles, 2022-12-26 23:00:00+00:00 2023-08-16 14:00:00+00:00
EUR_CHF H4 4000 candles, 2021-01-24 22:00:00+00:00 2023-08-16 09:00:00+00:00
EUR_NZD H1 4000 candles, 2022-12-26 22:00:00+00:00 2023-08-16 14:00:00+00:00
EUR_NZD H4 4000 candles, 2021-01-24 22:00:00+00:00 2023-08-16 09:00:00+00:00
EUR_CAD H1 4000 candles, 2022-12-26 23:00:00+00:00 2023-08-16 14:00:00+00:00
EUR_CAD H4 4000 candles, 2021-01-24 22:00:00+00:00 2023-08-16 09:00:00+00:00
EUR_AUD H1 4000 candles, 2022-12-26 22:00:00+00:00 2023-08-16 14:00:00+00:00